In [1]:
import basedosdados as bd
import pandas as pd
import os
import numpy as np

import geopandas as gpd 
import shapely
import pygeos
from shapely.wkt import loads
from shapely.geometry import Point, Polygon
import rtree

In [ ]:
#!pip install geopandas -q # Geopandas
#!pip install shapely -q #Shapely
#!pip install pygeos -q
#!pip install rtree -q

In [2]:
path = os.path.abspath(os.path.join('..', 'input'))
path_data = os.path.join(path, "tabela_cep.csv")
df = pd.read_csv(path_data, encoding='latin-1')
df

,cep,tipo,nome_logradouro,logradouro,bairro,cidade,estado,complemento,grandes_usuarios,tipo_sem_acento,...,bairro_sem_acento,cidade_sem_acento,complemento_sem_acento,grandes_usuarios_sem_acento,latitude,longitude,cidade_ibge,cidade_area,ddd,cep_ativo
0,1001000,Praça,da Sé,Praça da Sé,Sé,São Paulo,SP,lado ímpar,NaN,Praca,...,Se,Sao Paulo,lado impar,NaN,-23.550278,-46.634218,3550308,1521.110,11,S
1,1001001,Praça,da Sé,Praça da Sé,Sé,São Paulo,SP,lado par,NaN,Praca,...,Se,Sao Paulo,lado par,NaN,-23.549865,-46.634020,3550308,1521.110,11,S
2,1001010,Rua,Filipe de Oliveira,Rua Filipe de Oliveira,Sé,São Paulo,SP,NaN,NaN,Rua,...,Se,Sao Paulo,NaN,NaN,-23.551479,-46.633272,3550308,1521.110,11,S
3,1001900,Praça,"da Sé, 108","Praça da Sé, 108",Sé,São Paulo,SP,NaN,UNESP - Universidade Estadual Júlio de Mesquit...,Praca,...,Se,Sao Paulo,NaN,UNESP - Universidade Estadual Julio de Mesquit...,-23.549896,-46.634378,3550308,1521.110,11,S
4,1001901,Praça,"da Sé, 371","Praça da Sé, 371",Sé,São Paulo,SP,NaN,Edifício Santa Lídia,Praca,...,Se,Sao Paulo,NaN,Edificio Santa Lidia,-23.551829,-46.634234,3550308,1521.110,11,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1108500,99980000,NaN,NaN,NaN,NaN,David Canabarro,RS,NaN,NaN,NaN,...,NaN,David Canabarro,NaN,NaN,-28.411418,-51.831144,4306304,174.939,54,S
1108501,99980970,Rua,"Adelino Gazzoni, 160","Rua Adelino Gazzoni, 160",Centro,David Canabarro,RS,NaN,NaN,Rua,...,Centro,David Canabarro,NaN,NaN,-28.387671,-51.846788,4306304,174.939,54,S
1108502,99980974,Distrito,"Capela São José do Capingui, s/n","Distrito Capela São José do Capingui, s/n",Capela São José do Capingui (Distrito),David Canabarro,RS,NaN,NaN,Distrito,...,Capela Sao Jose do Capingui (Distrito),David Canabarro,NaN,NaN,-28.451355,-51.880714,4306304,174.939,54,S
1108503,99990000,NaN,NaN,NaN,NaN,Muliterno,RS,NaN,NaN,NaN,...,NaN,Muliterno,NaN,NaN,-28.343913,-51.746695,4312625,111.132,54,S


In [3]:
df_novo = df.loc[:,[  'cep',
                    'logradouro',
                    'bairro',
                    'cidade',
                    'complemento',
                    'cidade_ibge',
                    'estado',                    
                    'latitude',
                    'longitude'  ]]

df_novo.rename( columns = {'cep':'cep',
                        'logradouro':'logradouro',
                        'bairro':'bairro',
                        'cidade':'cidade',
                        'complemento':'complemento',
                        'cidade_ibge':'id_municipio',
                        'estado':'sigla_uf',
                        'latitude':'latitude',
                        'longitude':'longitude'}, inplace = True)

df_novo['cep'] = df_novo['cep'].astype(str)
df_novo['id_municipio'] = df_novo['id_municipio'].astype(str)

In [8]:
#Download do df. de CEP
query ='''
SELECT * FROM `basedosdados-dev.br_bd_diretorios_brasil.cep` 
'''

df = bd.read_sql(query, billing_project_id='dados-bd-359914')

Downloading: 100%|███████████████████████████████████████████████████████| 1108505/1108505 [03:34<00:00, 5176.90rows/s]


In [9]:
#Download da geometria de UF do GEOBR
query_geo = '''
SELECT * FROM `basedosdados.br_geobr_mapas.uf`
'''
x = bd.read_sql(query_geo, billing_project_id='dados-bd-359914')

Downloading: 100%|███████████████████████████████████████████████████████████████████| 27/27 [00:06<00:00,  3.87rows/s]


In [13]:
#Centróide em WKT
df['centroide'] = df['centroide'].apply(lambda x: loads(x))
df = gpd.GeoDataFrame(df, geometry='centroide')

#Transforma a geometria em WKT
x['geometria'] = x['geometria'].apply(lambda x: loads(x))
x = gpd.GeoDataFrame(x, geometry='geometria')

# Criar uma coluna "geometria_estado" no dataframe "df" para armazenar a geometria correspondente à sigla_uf de cada cep
df = df.merge(x[['sigla_uf', 'geometria']], on='sigla_uf', how='left')
df.rename(columns={'geometria': 'geometria_estado'}, inplace=True)

# Define o diretório de saída
output = os.path.abspath(os.path.join('..', 'output'))

# Obtém as siglas de todas as UF presentes no dataframe
ufs = list(df['sigla_uf'].unique())

# Itera por cada sigla de UF presente no dataframe
for uf in ufs:    
    # Filtra o dataframe para obter apenas as linhas com a sigla de UF correspondente
    estado = df[df['sigla_uf'] == uf].copy() 
    
    # Adiciona uma nova coluna 'pertence', que indica se o centroide (ponto médio) da unidade 
    # federativa está contido na geometria (polígono) da UF
    estado['pertence'] = estado.apply(lambda row: row['centroide'].within(row['geometria_estado']), axis=1)
    
    # Substitui as coordenadas latitude e longitude dos municípios que não estão contidos na UF
    # pelo valor None
    estado.loc[estado['pertence']== False, ['latitude', 'longitude']] = None
    
    # Remove as colunas que não serão usadas no arquivo de saída
    estado.drop(['pertence','centroide','geometria_estado','sigla_uf'], axis=1, inplace=True)
    
    # Cria a pasta de saída correspondente à UF, caso ela ainda não exista
    particao = output + f'/cep/sigla_uf={uf}/'
    if not os.path.exists(particao):
        os.makedirs(particao)
    
    # Define o caminho completo do arquivo CSV de saída e salva o dataframe nele
    particao = output + f'/cep/sigla_uf={uf}/cep.csv'
    estado.to_csv(particao, index=False, encoding='utf-8', na_rep='')
